In [ ]:
%config InlineBackend.figure_formats = ['svg']
import os

STATIC_WEB_PAGE = {"EXECUTE_NB", "READTHEDOCS"}.intersection(os.environ)

```{autolink-concat}
```

````{margin}
```{spec} Dalitz Distribution
:id: TR-018
:status: WIP
:tags: physics
```
````

# Dalitz Distribution

<!-- cspell:ignore coeff mmikhasenko msigma Remco -->

:::{epigraph}

Mikhail Mikhasenko [@mmikhasenko](https://github.com/mmikhasenko), Remco de Boer [@redeboer](https://github.com/redeboer)

:::

In [ ]:
%pip install -q ampform==0.14.0 qrules==0.9.7 sympy==1.10.1

This report is an attempt to formulate [this report](https://www.overleaf.com/7229968911cjshysdbfjtj) [behind login] on polarization sensitivity in $\Lambda_c \to p\pi K$ with [SymPy](https://docs.sympy.org) and [TensorWaves](https://tensorwaves.rtfd.io).

In [ ]:
from __future__ import annotations

import itertools
import logging
from functools import partial

import numpy as np
import qrules
import sympy as sp
from ampform.sympy import PoolSum, UnevaluatedExpression
from attrs import define
from IPython.display import Math, display
from qrules.particle import Particle, create_particle
from sympy.core.symbol import Str
from sympy.physics.quantum.cg import CG
from sympy.physics.quantum.spin import Rotation as Wigner

LOGGER = logging.getLogger()
LOGGER.setLevel(logging.ERROR)

PDG = qrules.load_pdg()


def display_definitions(definitions: dict[sp.Symbol, sp.Expr]) -> None:
    latex = R"\begin{array}{rcl}" + "\n"
    for symbol, expr in definitions.items():
        symbol = sp.sympify(symbol)
        expr = sp.sympify(expr)
        lhs = sp.latex(symbol)
        rhs = sp.latex(expr)
        latex += Rf"  {lhs} & = & {rhs} \\" + "\n"
    latex += R"\end{array}"
    display(Math(latex))


def display_doit(
    expr: UnevaluatedExpression, deep=False, terms_per_line: int = 10
) -> None:
    latex = sp.multiline_latex(
        lhs=expr,
        rhs=expr.doit(deep=deep),
        terms_per_line=terms_per_line,
        environment="eqnarray",
    )
    display(Math(latex))


# hack for moving Indexed indices below superscript of the base
def _print_Indexed_latex(self, printer, *args):
    base = printer._print(self.base)
    indices = ", ".join(map(printer._print, self.indices))
    return f"{base}_{{{indices}}}"


sp.Indexed._latex = _print_Indexed_latex

## Amplitude model

Naming convention: $\text{Baryon}(\mathbf{0}) \to \text{Baryon}(\mathbf{1}) P(\mathbf{2}) P(\mathbf{3})$
- **Chain 1**: $M^{23} \to P_2 P_3(23)$
- **Chain 2**: $B^{31} \to P_3\text{B}_1(31)$
- **Chain 3**: $B^{12} \to \text{B}_1P_2(12)$

where $B_1$ stands for a baryon with index (1) in the final state. $B^{31}$ and $B^{12}$ are baryon resonances.
The $M$ and $P$ denote the meson state, a general and pseudoscalar, respectively.

In what follows, the 'Chain 0' is the sum of the three chains.

In [ ]:
ΛcXX = create_particle(
  PDG["Lambda(c)(2625)+"], # bug in PDG!!!
  spin=3/2)
Λc = PDG["Lambda(c)+"]
πp = PDG["pi+"]
πm = PDG["pi-"]
# 
particles = [ΛcXX, Λc, πp, πm]
j0, j1, j2, j3 = list(map(lambda x: sp.Rational(x.spin), particles))
# 
chain_ids = {
    1: "pi^{**}",
    2: R"\Sigma_c^{**}",
    3: R"\Sigma_c^{**}",
}

Resonance choices and their $LS$-couplings are as follows:

In [ ]:
resonance_names = {
    1: ["rho(770)0"],
    2: ["Sigma(c)(2455)++"],
    3: ["Sigma(c)(2455)0"],
}
resonances = {
    chain_id: [PDG[name] for name in names]
    for chain_id, names in resonance_names.items()
}

### Aligned amplitude

In [ ]:
A_1 = sp.IndexedBase(R"A^{(23)}")
A_2 = sp.IndexedBase(R"A^{(31)}")
A_3 = sp.IndexedBase(R"A^{(12)}")

half = sp.S.Half

ζ_0_11 = sp.Symbol(R"\zeta^0_{1(1)}", real=True)
ζ_0_21 = sp.Symbol(R"\zeta^0_{2(1)}", real=True)
ζ_0_31 = sp.Symbol(R"\zeta^0_{3(1)}", real=True)
ζ_1_11 = sp.Symbol(R"\zeta^1_{1(1)}", real=True)
ζ_1_21 = sp.Symbol(R"\zeta^1_{2(1)}", real=True)
ζ_1_31 = sp.Symbol(R"\zeta^1_{3(1)}", real=True)

ζ_0_12 = sp.Symbol(R"\zeta^0_{1(2)}", real=True)
ζ_0_23 = sp.Symbol(R"\zeta^0_{2(3)}", real=True)
ζ_0_31 = sp.Symbol(R"\zeta^0_{3(1)}", real=True)
ζ_1_12 = sp.Symbol(R"\zeta^1_{1(2)}", real=True)
ζ_1_23 = sp.Symbol(R"\zeta^1_{2(3)}", real=True)
ζ_1_31 = sp.Symbol(R"\zeta^1_{3(1)}", real=True)


m1, m2, m3 = sp.symbols(R"m_1 m_2 m_3", nonnegative=True)

def formulate_aligned_amplitude(λ0, λ1):
    _ν = sp.Symbol(R"\nu^{\prime}", rational=True)
    _λ = sp.Symbol(R"\lambda^{\prime}", rational=True)
    return PoolSum(
        A_1[_ν, _λ]
        * Wigner.d(j0, λ0, _ν, ζ_0_11)
        * Wigner.d(j1, _λ, λ1, ζ_1_11)
        + A_2[_ν, _λ]
        * Wigner.d(j0, λ0, _ν, ζ_0_21)
        * Wigner.d(j1, _λ, λ1, ζ_1_21)
        + A_3[_ν, _λ]
        * Wigner.d(j0, λ0, _ν, ζ_0_31)
        * Wigner.d(j1, _λ, λ1, ζ_1_31),
        (_λ, create_spin_range(j1)),
        (_ν, create_spin_range(j0)),
    )


ν = sp.Symbol("nu")
λ = sp.Symbol("lambda")
formulate_aligned_amplitude(λ0=ν, λ1=λ)

### Decay chain amplitudes

Example of a data container for all decay info. More info on this syntax [here](https://www.attrs.org/en/stable/examples.html):

In [ ]:
@define
class Decay:
    j: int
    LS: int
    ls: int
    chain_ind: int

In [ ]:
def formulate_chain_amplitude(chain_id: int, λ0, λ1):
    resonances_in_this_subsystem = resonances[chain_id]
    if chain_id == 1:
        return formulate_23_amplitude(λ0, λ1, resonances_in_this_subsystem)
    if chain_id == 2:
        return formulate_31_amplitude(λ0, λ1, resonances_in_this_subsystem)
    if chain_id == 3:
        return formulate_12_amplitude(λ0, λ1, resonances_in_this_subsystem)
    raise NotImplementedError


H_prod = sp.IndexedBase(R"\mathcal{H}")
H_dec = sp.IndexedBase(R"\mathcal{K}")

θ23 = sp.Symbol("theta23", real=True)
θ31 = sp.Symbol("theta31", real=True)
θ12 = sp.Symbol("theta12", real=True)

σ1, σ2, σ3 = sp.symbols("sigma1:4", nonnegative=True)

R_1 = sp.Symbol(R"\mathcal{R}^{\pi}", real=True)
R_2 = sp.Symbol(R"\mathcal{R}^{\Sigma_c^{++}}", real=True)
R_3 = sp.Symbol(R"\mathcal{R}^{\Sigma_c^{0}}", real=True)
Rs = {1: R_1, 2: R_2, 3: R_3}


def formulate_23_amplitude(λ0, λ1, resonances: list[Particle]):
    τ = sp.Symbol("tau", rational=True)
    return sp.Add(
        *[
            PoolSum(
                sp.KroneckerDelta(λ0, τ - λ1)
                * H_prod[Str(res.latex), τ, λ1]
                * (-1) ** (j1 - λ1)
                * R_1
                * Wigner.d(sp.Rational(res.spin), τ, 0, θ23)
                * H_dec[Str(res.latex), 0, 0],
                (τ, create_spin_range(res.spin)),
            )
            for res in resonances
        ]
    )


def formulate_31_amplitude(λ0, λ1, resonances: list[Particle]):
    τ = sp.Symbol("tau", rational=True)
    return sp.Add(
        *[
            PoolSum(
                sp.KroneckerDelta(λ0, τ)
                * H_prod[Str(res.latex), τ, 0]
                * Wigner.d(sp.Rational(res.spin), τ, -λ1, θ31)
                * R_2
                * H_dec[Str(res.latex), 0, λ1]
                * (-1) ** (j1 - λ1),
                (τ, create_spin_range(res.spin)),
            )
            for res in resonances
        ]
    )


def formulate_12_amplitude(λ0, λ1, resonances: list[Particle]):
    τ = sp.Symbol("tau", rational=True)
    return sp.Add(
        *[
            PoolSum(
                sp.KroneckerDelta(λ0, τ)
                * H_prod[Str(res.latex), τ, 0]
                * Wigner.d(sp.Rational(res.spin), τ, λ1, θ12)
                * R_3
                * H_dec[Str(res.latex), λ1, 0],
                (τ, create_spin_range(res.spin)),
            )
            for res in resonances
        ]
    )


def create_spin_range(j):
    return arange(-j, +j)


def arange(x1, x2):
    spin_range = np.arange(float(x1), +float(x2) + 0.5)
    return list(map(sp.Rational, spin_range))


display(
    formulate_chain_amplitude(1, ν, λ),
    formulate_chain_amplitude(2, ν, λ),
    formulate_chain_amplitude(3, ν, λ),
)

### Helicity coupling values

In [ ]:
parameter_defaults = {}

In [ ]:
dec_couplings = {}
for res in resonances[1]:
    dec_couplings[H_dec[Str(res.latex), 0, 0]] = 1
for res in resonances[2]:
    dec_couplings[H_dec[Str(res.latex), 0, half]] = 1
    dec_couplings[H_dec[Str(res.latex), 0, -half]] = (
        int(res.parity)
        * int(particles[3].parity)
        * int(particles[1].parity)
        * (-1) ** (res.spin - j3 - j1)
    )
for res in resonances[3]:
    dec_couplings[H_dec[Str(res.latex), half, 0]] = 1
    dec_couplings[H_dec[Str(res.latex), -half, 0]] = (
        int(res.parity)
        * int(particles[1].parity)
        * int(particles[2].parity)
        * (-1) ** (res.spin - j1 - j2)
    )
parameter_defaults.update(dec_couplings)
display_definitions(dec_couplings)

The production couplings are written in the LS basis:
$$
\mathcal{H}_{\lambda,\lambda'} = \sum_{LS}
    \left\langle j,\lambda ; j',\lambda' | S, \lambda-\lambda'\right\rangle
    \left\langle L,0 ; S,\lambda-\lambda' | J, \lambda-\lambda'\right\rangle \mathcal{H}_{LS}
$$

In [ ]:
def recoupling(λ1, λ2, j, l, s, j1, j2):
    Δλ = λ1 - λ2
    return CG(j1, λ1, j2, -λ2, s, Δλ) * CG(l, sp.S(0), s, Δλ, j, Δλ)

In [ ]:
prod_couplings = {}
LSs = {1: (sp.S(0), 3*half), 2: (sp.S(2), half), 3: (sp.S(2), half)}
j0 = sp.Rational(ΛcXX.spin)
j3_values = {1: j1, 2: j2, 3: j3}
for chain_id in [1, 2, 3]:
    for res in resonances[chain_id]:
        l, s = LSs[chain_id]
        j3 = j3_values[chain_id]
        for λ1 in create_spin_range(res.spin):
            for λ2 in create_spin_range(j3):
                c = recoupling(
                    sp.Rational(λ1),
                    sp.Rational(λ2),
                    sp.Rational(j0),
                    l,
                    s,
                    sp.Rational(res.spin),
                    sp.Rational(j3),
                )
                if c.doit() != 0:
                    prod_couplings[
                        H_prod[
                            Str(res.latex), sp.Rational(λ1), sp.Rational(λ2)
                        ]
                    ] = c
display_definitions(prod_couplings)
couplings = dict(dec_couplings)
couplings.update(prod_couplings)
parameter_defaults.update(prod_couplings)

### Intensity expression

In [ ]:
def formulate_intensity(amplitude_builder):
    return PoolSum(
        sp.Abs(amplitude_builder(λ, ν)) ** 2,
        (λ, create_spin_range(j1)),
        (ν, create_spin_range(j0)),
    )


intensity_expressions = {
    0: formulate_intensity(formulate_aligned_amplitude),
    1: formulate_intensity(partial(formulate_chain_amplitude, 1)),
    2: formulate_intensity(partial(formulate_chain_amplitude, 2)),
    3: formulate_intensity(partial(formulate_chain_amplitude, 3)),
}
intensity_expressions[0]

In [ ]:
A = {1: A_1, 2: A_2, 3: A_3}
amp_definitions = {}
for chain_id in chain_ids:
    for heli0, heli1 in itertools.product(create_spin_range(j0), create_spin_range(j1)):
        symbol = A[chain_id][heli0, heli1]
        expr = formulate_chain_amplitude(chain_id, ν, λ)
        amp_definitions[symbol] = expr.subs({ν: heli0, λ: heli1})
display_definitions(amp_definitions)

Now we put all equations together and compute the sum over the indices

In [ ]:
expr = (
    formulate_intensity(formulate_aligned_amplitude)
    .doit()
    .xreplace(amp_definitions)
)
up_to_couplings = expr.doit()
final = up_to_couplings.xreplace(couplings).doit()
final_expanded = sp.expand(final)

The relations of the Wigner angles are used to simplify the final expressions.
One expects that 
 - there are no wigner angles for the intensity of any isobar.
 - the $(ij)$-interference term might depend only on the $\zeta_{i(j)}$ angle.

In [ ]:
simplified_intensities = {}
Ts = sp.IndexedBase(R"\mathcal{T}")
# diagonal terms
for i in range(1, 4):
    simplified_intensities[Ts[i, i]] = sp.simplify(
        final_expanded.coeff(Rs[i], 2)
    )
# off-diagonal terms
for ij in {12, 23, 31}:
    i, j = divmod(ij, 10)
    coefficient = final_expanded.coeff(Rs[i] * Rs[j], 1)
    simplified_intensities[Ts[i, j]] = sp.simplify(coefficient / sp.S(2))
#

The total intensity is written a bilinear for on dynamic functions $\mathcal{R}$ with coefficients $T_{ij}$ describing the angular dependence.

In [ ]:
intensity_sum = sum(
    Ts[i, j] * Rs[i] * sp.conjugate(Rs[j])
    for i in range(1, 4)
    for j in range(1, 4)
)
intensity_sum = intensity_sum.subs(
    {Ts[2, 1]: Ts[1, 2], Ts[1, 3]: Ts[3, 1], Ts[3, 2]: Ts[2, 3]}
)
display_definitions({sp.Symbol(R"\mathcal{I}_\text{tot}"): intensity_sum})

In [ ]:
afactor = 60
display_definitions({
    Ts[1, 1]: afactor * simplified_intensities[Ts[1, 1]], # T(ii) can never have zeta angles
    Ts[2, 2]: afactor * simplified_intensities[Ts[2, 2]],
    Ts[3, 3]: afactor * simplified_intensities[Ts[3, 3]],
    Ts[1, 2]: afactor * simplified_intensities[Ts[1, 2]].subs({
        ζ_0_11: 0,
        ζ_1_11: 0,
    }),
    Ts[2, 3]: afactor * simplified_intensities[Ts[2, 3]].subs({
        ζ_0_11: 0,
        ζ_1_11: 0,
        ζ_0_21: -2 * sp.pi + ζ_0_23 + ζ_0_31,
        ζ_1_21: ζ_1_23 + ζ_1_31,
    }),
    Ts[3, 1]: afactor * simplified_intensities[Ts[1, 2]].subs({
        ζ_0_11: 0,
        ζ_1_11: 0,
    })
})

Something is fishy in with the final expression since the diagonal terms cannot have the the wigner angels.